Copyright 2023 The TensorFlow Authors.


In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tutorial de pipeline TFX para LLM usando o dataset CNN Daily

Neste codelab, usaremos o KerasNLP para carregar um Large Language Model (LLM) pré-treinado, modelo GPT-2, e ajustá-lo a um dataset. O dataset usado nesta demonstração é o dataset CNN Daily. Observe que o GPT-2 é usado aqui apenas para demonstrar o processo de ponta a ponta; as técnicas e ferramentas apresentadas neste codelab são potencialmente transferíveis para outros modelos de linguagem generativa, como o Google T5.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/CSV_Downloader_Component"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/CSV_Downloader_Component.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

# Antes de começar

O Colab oferece diferentes tipos de sistemas de runtime. Certifique-se de acessar **Runtime -&gt; Change runtime type** e escolher o runtime GPU Hardware Accelerator (que deve ter mais de 12GB de RAM de sistema e aproximadamente 15GB de RAM de GPU), pois você fará ajustes finos no modelo GPT-2.

# Configuração

Primeiro instalaremos o pacote TFX Python.

## Atualize o Pip

Para evitar a atualização do Pip num sistema ao executar localmente, garanta que estamos executando no Colab. Os sistemas locais podem, claro, ser atualizados separadamente.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

## Instale o TFX

Atualmente, o TFX está tendo alguns problemas com Python 3.10 no Colab. Portanto, simplesmente executar o comando

```
!pip install -U tfx
```

para instalar o tfx vai **falhar**. Portanto, siga o código abaixo.

In [ ]:
%%shell
update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.8 3
curl -O https://bootstrap.pypa.io/get-pip.py
python get-pip.py

In [ ]:
# 1) TFX relies on an old version of google-api-core so we let google-auth float
# for the install. We grep it out below:
!grep -v google-auth /etc/requirements.core.in > requirements.txt

# 2) httplib2 should be included in /etc/requirements.core.in but it's not for
# reasons. We ensure it's included:
!grep httplib2 /etc/requirements.user.in >> requirements.txt

# 3) google.colab package is not available as a wheel. We symlink that in so
# it's on the sys.path of Python 3.8:
!mkdir /usr/local/lib/python3.8/dist-packages/google
!ln -s /usr/local/lib/python3.10/dist-packages/google/colab /usr/local/lib/python3.8/dist-packages/google/colab

# Now with those pre-requisites out of the way:
!pip install tfx==1.13.0 -r requirements.txt

In [ ]:
!pip install keras_nlp

# Importações

Vamos primeiro tirar nossas importações do caminho.

In [ ]:
from tensorflow import keras
from tfx.types import Channel
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

## Desinstale o shapely

TODO(b/263441833) Esta é uma solução temporária para evitar um ImportError. Em última análise, isto deverá ser resolvido com suporte a uma versão mais recente do Bigquery, em vez de desinstalar outras dependências extras.


In [ ]:
!pip uninstall shapely -y

## Você reiniciou o runtime?

Se você estiver usando o Google Colab, na primeira vez que executar a célula acima, você deve reiniciar o runtime clicando no botão "RESTART RUNTIME" acima ou usando o menu "Runtime &gt; Restart runtime ...". Isso é necessário devido à maneira como o Colab carrega os pacotes.

Verifique as versões do TensorFlow e TFX.

Vamos verificar as versões das bibliotecas.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

## Configuração de variáveis

Existem algumas variáveis ​​usadas para definir um pipeline. Você pode personalizar essas variáveis ​​como desejar. Por padrão, toda a saída do pipeline será gerada no diretório atual.

# Componente CSV Downloader

Para tornar o pipeline mais eficiente e possível de automação, é útil ter um componente que receba um link de download para o arquivo CSV a ser baixado. Além disso, um objetivo importante do pipeline de ML em produção do TFX é coletar metadados contendo informações sobre os componentes do pipeline, suas execuções e artefatos resultantes. Em outras palavras, o objetivo dos metadados é analisar a linhagem dos componentes do pipeline e depurar problemas, e o componente CSV Downloader ajudaria os usuários a registrar e rastrear informações sobre a origem dos dados e as etapas de pré-processamento pelas quais os dados foram submetidos antes de entrar no pipeline. Nesta seção, declaramos um novo artefato chamado CSVdoc e desenvolvemos um componente personalizado – o CSV Downloader – que armazena informações sobre o dataset e baixa o arquivo CSV na URI do artefato CSVdoc.

In [ ]:
from tfx.types import artifact
from tfx import types

Property = artifact.Property
PropertyType = artifact.PropertyType

URL_PROPERTY = Property(type=PropertyType.STRING)
PATH_PROPERTY = Property(type=PropertyType.STRING)

class CsvDoc(types.Artifact):
  """ Artifact that contains the CSV dataset.

     - 'url' : saves the source of the original data.
     - 'path': saves the path to the CSV file.
  """

  TYPE_NAME = 'CsvDoc'
  PROPERTIES = {
      'url' : URL_PROPERTY,
      'path': PATH_PROPERTY,
  }

In [ ]:
from absl import logging
import requests
import os
import tfx.v1 as tfx
from tfx.dsl.component.experimental.decorators import component

@tfx.dsl.components.component
def CsvDownloaderComponent(
    url: tfx.dsl.components.Parameter[str],
    file_name: tfx.dsl.components.Parameter[str],
    saved_file: tfx.dsl.components.OutputArtifact[CsvDoc],
) -> None:
  response = requests.get(url)
  saved_file.url = url
  if response.status_code == 200:
    file_path = os.path.join(saved_file.uri, file_name)
    saved_file.path = file_path
    url_content = response.content
    with open(file_path, 'wb') as csv_file:
      csv_file.write(url_content)
    logging.info(f"CSV file saved successfully at {file_path}")
  else:
    raise Exception("CSV file failed to be saved.")

In [ ]:
downloader = CsvDownloaderComponent(
  url = 'https://drive.google.com/uc?id=1YdZsJlRafqxiNSl0nHQkwR7rzrNlN9LI&export=download', file_name ='testing_doc.csv')

In [ ]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
context = InteractiveContext()

In [ ]:
context.run(downloader, enable_cache = False)